In [1]:
import cv2
import sys
import numpy as np
from cv2 import imshow as show

In [2]:
import tensorflow as tf
from tensorflow import keras

In [3]:
import tensorflow.keras.applications.vgg16  as vgg
from tensorflow.keras import Model
from tensorflow.keras.layers import Layer,Flatten,Dropout,Dense
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras import layers, models, Model, optimizers
from sklearn.model_selection import train_test_split as tts

In [5]:
cellStates = ['empty','present','absent']

In [6]:
# to rotate an image 
def rotate(img, angle):
    rows = img.shape[0]
    cols = img.shape[1]

    img_center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(img_center, angle, 1)

    rotated_image = cv2.warpAffine(img, M, (cols, rows),
                    borderMode=cv2.BORDER_CONSTANT,
                    borderValue=(255,255,255))
    return rotated_image

In [9]:
# Color conversion
def getGreyScaleImage( img ): 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  
#     cv2.imshow("grey", img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return img

In [10]:
# Image Thresholding
def getThreshImage( img ):  
    ret, thresh = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU )
#     cv2.imshow("thresh", thresh)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return thresh

In [11]:
# Morphological Transformation
def getErosionImage( img ):
    kernel = np.ones((5,5),np.uint8)
    erosion = cv2.erode(img,kernel,iterations = 1)
#     cv2.imshow("closing", erosion)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return erosion

In [25]:
# Invert binarize the image
def invertImage( img ):
    img_blur = cv2.GaussianBlur(img, (3,3), 0)
    canny_edges = cv2.Canny(img_blur, 10, 70)
    ret, mask = cv2.threshold(canny_edges, 70, 255, cv2.THRESH_BINARY_INV)
    return mask

In [13]:
# checking the functions
img = cv2.imread('handA.png')
erosion = getErosionImage(img)
grey = getGreyScaleImage(img)
thresh = getThreshImage(grey)
cv2.imshow("img", invertImage(img))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
#define detecting the edges
def edgeDetection( img ):
    # Convert to graycsale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)
    # Sobel Edge Detection on the X axis
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=1) 
     # Sobel Edge Detection on the Y axis
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=1)
    # Combined X and Y Sobel Edge Detection
    sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=1) 
    # Display Sobel Edge Detection Images
#     cv2.imshow('Sobel X', sobelx)
#     cv2.waitKey(0)
#     cv2.imshow('Sobel Y', sobely)
#     cv2.waitKey(0)
#     cv2.imshow('Sobel X Y using Sobel() function', sobelxy)
#     cv2.waitKey(0)
    
    # Canny Edge Detection
    edges = cv2.Canny(image=img_blur, threshold1=20, threshold2=70)
    # Display Canny Edge Detection Image
#     cv2.imshow('Canny Edge Detection', edges)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    return invertImage(edges)

In [15]:
#checking the 
img = cv2.imread('handA.png') 
img = edgeDetection(img)
img = invertImage(img)
cv2.imshow("img", (img))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
# video cam detection
cam = cv2.VideoCapture(0)

while True :
    ret, frame = cam.read()
    cv2.imshow('Live', edgeDetection(frame))
    if( cv2.waitKey(1) == 13 ): #enter key
        break

cam.release()
cv2.destroyAllWindows()